In [1]:
import pandas as pd
import re

In [6]:
atlas = pd.read_csv('atlas_all_reactions.txt',sep=';',header=None)
atlas.columns = ['id','rn','atlas_equation','kegg_equation','ec','delta_G','delta_G_error','kegg_match','score']

In [62]:
x = atlas.kegg_equation.tolist()[0]

In [94]:
x = atlas[atlas['kegg_equation'].apply(lambda x: len(re.findall('\(\d\)(?=C)',x))) > 0].iloc[0].kegg_equation

In [55]:
example = re.search('\d(?=C)','2C00002+C00003<=>C00004+C00080+2C07024')

In [132]:
def parse_reaction_string(x):
    reaction_dict = {}
    # split to reactions and produts
    subs_prods = x.split('<=>')
    subs = subs_prods[0]
    prods = subs_prods[1]
    subs = subs.split('+')
    prods = prods.split('+')
    # separate
    for sub  in subs:
        stoich = re.findall('(?<=\()\d*(?=\))',sub)
        if len(stoich) > 0:
            s = int(stoich[0])
            compound = sub.split(')')[1]
            reaction_dict[compound] = -s
        else:
            reaction_dict[sub] = -1
            
    for prod  in prods:
        stoich = re.findall('(?<=\()\d*(?=\))',prod)
        if len(stoich) > 0:
            s = int(stoich[0])
            compound = prod.split(')')[1]
            reaction_dict[compound] = s
        else:
            reaction_dict[prod] = 1       
    return reaction_dict

In [133]:
dfs = []

for idx,row in atlas.iterrows():
    x = row.kegg_equation
    df = pd.DataFrame(pd.Series(parse_reaction_string(x)),columns = ['s']).reset_index()
    rn = 'AR' + str(row.id)
    df.columns = ['cid','s']
    df['rn']  = rn
    dfs.append(df)

In [134]:
atlas_network = pd.concat(dfs,axis=0)

In [141]:
adg = atlas[['id','delta_G','delta_G_error']].dropna()
adg['rn'] = adg.id.apply(lambda x: 'AR' + str(x))
adg = adg.set_index('rn')[['delta_G','delta_G_error']]

In [152]:
atlas_network[['rn','cid','s']].set_index('rn').to_csv('atlas.network.v1.2020.csv')
adg.to_csv('atlas.deltaG.v1.2020.csv')

In [155]:
import networkExpansionPy.lib as ne

In [156]:
thermo = pd.read_csv(ne.asset_path +'/reaction_free_energy/kegg_reactions_CC_ph7.0.csv',sep=',')

In [163]:
atlas['atlas_rn'] = atlas.id.apply(lambda x: 'AR' + str(x))


In [167]:
atlas[['atlas_rn','rn']].dropna().to_csv('atlas2kegg.csv')

In [174]:
atlas[atlas.id == 10306]

,id,rn,atlas_equation,kegg_equation,ec,delta_G,delta_G_error,kegg_match,score,atlas_rn
10305,10306,NaN,(4)50+0+2+83+6572<=>3+69,(4)C00059+C00001+C00003+C00094+C13768<=>C00004...,1.8.1.-(rev),NaN,NaN,R08356/1.8.1.12,0.238352,AR10306
